# Experiment 4 on real datasets.
**Authors:**
* José Ángel Martín-Baos
* Julio Alberto López-Gomez
* Luis Rodríguez-Benítez
* Tim Hillel
* Ricardo García-Ródenas

## Imports and function definitions

In [16]:
%load_ext autoreload
%autoreload 2
## Import packages
import pandas as pd  # For file input/output
from scipy import optimize
from scipy.optimize._numdiff import approx_derivative
import sys

# append a new directory to sys.path
sys.path.append('../rumboost and RUMs')

import time
import numpy as np
import pickle
import copy
import gc
#import matplotlib.pyplot as plt
#import shap
import lightgbm as lgb
import hyperopt
import warnings
#import torch
from datasets import load_preprocess_SwissMetro
from rumbooster import rum_train
from utils import bio_to_rumboost
from models import LPMC, LPMC_normalised, LPMC_nested_normalised, SwissMetro, SwissMetro_normalised
from benchmarks import return_dataset, prepare_model, estimate_models, prepare_labels, predict_test, predict_proba
from models import SwissMetro

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, accuracy_score

from sklearn.model_selection import GroupShuffleSplit

# Load common functions for the experiments
from expermients_functions import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
dataset = load_preprocess_SwissMetro(full_data=True)

In [6]:
dataset

,ID,TRAIN_TT,TRAIN_COST,TRAIN_HE,SM_TT,SM_COST,SM_HE,CAR_TT,CAR_CO,MALE,...,PURPOSE_2,PURPOSE_3,PURPOSE_4,PURPOSE_5,PURPOSE_6,PURPOSE_7,PURPOSE_8,AGE_1,AGE_2,choice
0,1,112,48,120,63,52,20,117,65,0,...,0,0,0,0,0,0,0,0,0,1
1,1,103,48,30,60,49,10,117,84,0,...,0,0,0,0,0,0,0,0,0,1
2,1,130,48,60,67,58,30,117,52,0,...,0,0,0,0,0,0,0,0,0,1
3,1,103,40,30,63,52,20,72,52,0,...,0,0,0,0,0,0,0,0,0,1
4,1,130,36,60,63,42,20,90,84,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,1192,148,13,30,93,17,30,156,56,1,...,0,0,1,0,0,0,0,0,0,1
10724,1192,148,12,30,96,16,10,96,70,1,...,0,0,1,0,0,0,0,0,0,2
10725,1192,148,16,60,93,16,20,96,56,1,...,0,0,1,0,0,0,0,0,0,2
10726,1192,178,16,30,96,17,30,96,91,1,...,0,0,1,0,0,0,0,0,0,1


In [5]:
dataset.ID.unique().shape

(1004,)

In [18]:
## Import the Classification models
# from Models.MNL import MNL
# from Models.SVM import SVM
# from Models.RandomForest import RandomForest
from Models.LightGBM import LightGBM
from Models.NN import NN
from Models.DNN import DNN
from Models.ResLogit import ResLogit
#from Models.CNN import CNN
# from Models.ResNet import ResNet

In [19]:
# Customize matplotlib
tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True, 
    "font.family": "serif",
    # Use 14pt font in plots, to match 10pt font in document
    "axes.labelsize": 14,
    "font.size": 14,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
}

#plt.rcParams.update(tex_fonts)

## Experiment initialization

### Experiment parameters

In [33]:
## Experiment parameters
data_dir = "../Data/Datasets/preprocessed/"
adjusted_hyperparms_dir = "../Data/adjusted-hyperparameters/"
train_suffix = "_train.csv"
test_suffix = "_test.csv"
hyperparameters_suffix = "_hyperparameters"
reset_crossval_indices = 0 # Set to 0 for reproducibility of the experiment over multiple executions
partial_results_dir = "../Data/Results-RealDatasets/"

recompute_Experiment_4 = False

rounding = 2

CV = 5 # Number of cross-validation
n_iter = 50 #  Number of iterations used on the random search
n_i = 20 # Number of iterations for the train test split
average_tech = "macro" #"micro"

hyperparameters_suffix = hyperparameters_suffix +'_'+ str(n_iter) + '.csv'

# Set the hyperparameters search space
hyperparameters = {"NN"  : {"hidden_layer_sizes": hyperopt.pyll.scope.int(hyperopt.hp.quniform('hidden_layer_sizes', 10, 100,1)),
                            "activation" : hyperopt.hp.choice('activation', ["tanh"]), # TODO: Consider other activation functions (ReLU, LeakyReLU, etc.)
                            "solver" : hyperopt.hp.choice('solver', ["lbfgs","sgd","adam"]),
                            "learning_rate_init": hyperopt.hp.uniform('learning_rate_init', 0.0001, 1),
                            "learning_rate" : hyperopt.hp.choice('learning_rate', ["adaptive"]),
                            "max_iter": hyperopt.hp.choice('max_iter', [10000000]),
                            "batch_size": hyperopt.hp.choice('batch_size', [128,256,512,1024]),
                            "tol" : hyperopt.hp.choice('tol', [1e-3]),
                           },
                   "DNN"  : {"input_dim": hyperopt.hp.choice('input_dim', [20]),
                             "output_dim": hyperopt.hp.choice('output_dim', [3]),
                             "depth": hyperopt.hp.choice('depth', [2,3,4,5,6]),
                             "width": hyperopt.hp.choice('width', [25,50,100]),
                             "drop": hyperopt.hp.choice('drop', [0.5, 0.3, 0.1]),
                             # TODO: Consider adding the activation functions for the hidden layers (thanh, ReLU, LeakyReLU, etc.)
                             "epochs": hyperopt.pyll.scope.int(hyperopt.hp.quniform('epochs', 50, 200,1)),
                             "batch_size": hyperopt.hp.choice('batch_size', [128,256,512,1024]),
                           },
                   "LightGBM" : {'num_leaves': hyperopt.pyll.scope.int(hyperopt.hp.quniform('num_leaves', 2, 100, 1)),
                                'min_gain_to_split':  hyperopt.hp.loguniform('min_gain_to_split', -9.21, 1.61), # interval: [0.0001, 5]
                                'min_sum_hessian_in_leaf': hyperopt.hp.loguniform('min_sum_hessian_in_leaf', 0, 4.6),
                                'min_data_in_leaf': hyperopt.pyll.scope.int(hyperopt.hp.quniform('min_data_in_leaf', 1, 200, 1)),
                                'bagging_fraction': hyperopt.hp.uniform('bagging_fraction', 0.5, 1),
                                'bagging_freq': hyperopt.hp.choice('bagging_freq', [1, 5, 10]),
                                'feature_fraction': hyperopt.hp.uniform('feature_fraction', 0.5, 1),
                                'lambda_l1': hyperopt.hp.loguniform('lambda_l1', -9.21, 2.30), # interval: [0.0001, 10]
                                'lambda_l2': hyperopt.hp.loguniform('lambda_l2', -9.21, 2.30), # interval: [0.0001, 10]
                                'max_bin': hyperopt.pyll.scope.int(hyperopt.hp.quniform('max_bin', 100, 500, 1)),
                                'num_iterations': hyperopt.pyll.scope.int(hyperopt.hp.quniform('num_iterations', 1, 3000, 1))
                            },
                   "RUMBoost" : {'num_leaves': hyperopt.pyll.scope.int(hyperopt.hp.quniform('num_leaves', 2, 100, 1)),
                                'min_gain_to_split':  hyperopt.hp.loguniform('min_gain_to_split', -9.21, 1.61), # interval: [0.0001, 5]
                                'min_sum_hessian_in_leaf': hyperopt.hp.loguniform('min_sum_hessian_in_leaf', 0, 4.6),
                                'min_data_in_leaf': hyperopt.pyll.scope.int(hyperopt.hp.quniform('min_data_in_leaf', 1, 200, 1)),
                                'bagging_fraction': hyperopt.hp.uniform('bagging_fraction', 0.5, 1),
                                'bagging_freq': hyperopt.hp.choice('bagging_freq', [1, 5, 10]),
                                'feature_fraction': hyperopt.hp.uniform('feature_fraction', 0.5, 1),
                                'lambda_l1': hyperopt.hp.loguniform('lambda_l1', -9.21, 2.30), # interval: [0.0001, 10]
                                'lambda_l2': hyperopt.hp.loguniform('lambda_l2', -9.21, 2.30), # interval: [0.0001, 10]
                                'max_bin': hyperopt.pyll.scope.int(hyperopt.hp.quniform('max_bin', 100, 500, 1))
                            },
                    "ResLogit":{"input_dim": hyperopt.hp.choice('input_dim', [20]),
                             "output_dim": hyperopt.hp.choice('output_dim', [3]),
                             "depth": hyperopt.hp.choice('depth', [4, 8, 16, 32]),
                             #"drop": hyperopt.hp.choice('drop', [0.5, 0.3, 0.1]),
                             # TODO: Consider adding the activation functions for the hidden layers (thanh, ReLU, LeakyReLU, etc.)
                             "epochs": hyperopt.pyll.scope.int(hyperopt.hp.quniform('epochs', 200, 201,1)),
                             "batch_size": hyperopt.hp.choice('batch_size', [128,256,512,1024]),
                           },
                   }

scaled_fetures = ['TRAIN_TT', 'SM_TT', 'CAR_TT', 'TRAIN_COST','TRAIN_HE','SM_COST','SM_HE','CAR_CO']

model_type_to_class = {"LightGBM": LightGBM,
                       "NN": NN,
                       "DNN": DNN,
                       "RUMBoost": "RUMBoost",
                       "MNL":"MNL",
                       #"ResLogit":"ResLogit",
                      }

STATIC_PARAMS = {'n_jobs': -1,
                 'num_classes':3,
                 'objective':'multiclass',
                 'boosting': 'gbdt',
                 'monotone_constraints_method': 'advanced',
                 'verbosity': -1,
                 'early_stopping_round': 100,
                 'num_iterations': 3000,
                 'learning_rate': 0.1,
                 'max_depth':1,
                 'max_bin': 255,
                 'min_data_in_leaf': 20,
                 'min_sum_hessian_in_leaf': 0.0001,
                 'min_data_in_bin': 3
                 }

In [206]:
def objective(space):
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)  # Ignore deprecation warnings

    # Create the classifier
    if classifier == "RUMBoost":
        params = {**space, **STATIC_PARAMS}
        params['max_depth'] = 1
        rum_structure = bio_to_rumboost(models_train[0])
    elif classifier == "ResLogit":
        pass
    else:
        params = {**space}
        clf = model_type_to_class[classifier](**params)

    # Applying k-Fold Cross Validation
    loss = 0
    N_sum = 0

    for iteration in range(0, len(train_indices)):
        # Obtain training and testing data for this iteration (split of de k-Fold)
        X_train_CV, X_test_CV = X_train.iloc[train_indices[iteration]], X_train.iloc[test_indices[iteration]]
        y_train_CV, y_test_CV = y_train.iloc[train_indices[iteration]], y_train.iloc[test_indices[iteration]]

        # Scale the data
        scaler = StandardScaler()
        scaler.fit(X_train_CV[scaled_fetures])
        X_train_CV.loc[:, scaled_fetures] = scaler.transform(X_train_CV[scaled_fetures])
        X_test_CV.loc[:, scaled_fetures] = scaler.transform(X_test_CV[scaled_fetures])

        if classifier == "RUMBoost":
            train_set = lgb.Dataset(X_train_CV, label=y_train_CV, free_raw_data=False)
            test_set = lgb.Dataset(X_test_CV, label=y_test_CV, free_raw_data=False)
            clf_trained = rum_train(params, train_set, rum_structure, valid_sets=[test_set])
            loss += clf_trained.best_score * X_test_CV.shape[0]
            N_sum += X_test_CV.shape[0]

        elif classifier == "ResLogit":
            clf = ResLogit(X_train_CV, y_train_CV, space["input_dim"], space["output_dim"], n_layers=space['depth'], batch_size=space['batch_size'], epochs=space['epochs'])
            loss_it, _, _ = clf.train(X_train_CV, y_train_CV, X_test_CV, y_test_CV)
            loss += loss_it * X_test_CV.shape[0]
            N_sum += X_test_CV.shape[0]

        else:
            clf.fit(X_train_CV, y_train_CV)

            proba = clf.predict_proba(X_test_CV)

            # Cross-Entropy Loss
            sum = 0
            i = 0
            for sel_mode in y_test_CV.values:
                sum = sum + np.log(proba[i,sel_mode])
                i += 1
            N = i - 1
            loss += -sum  # Original: (-sum/N) * N
            N_sum += N

    loss = loss / N_sum
    return {'loss': loss, 'status': hyperopt.STATUS_OK}

### Load the data

In [20]:
datasets = {"LPMC": {
                  "name": "LPMC",
                  "mode_var": "travel_mode",
                  "individual_id": "household_id",
                  "scaled_fetures": ['day_of_week', 'start_time_linear', 'age', 'car_ownership',
                                     'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail',
                                     'dur_pt_bus', 'dur_pt_int_waiting', 'dur_pt_int_walking', 'pt_n_interchanges',
                                     'dur_driving', 'cost_transit', 'cost_driving_fuel'],
                  "alt_names": ["Walk", "Bike", "Public transport", "Car"]
                }
}

In [21]:
## Create the classifier
def create_classifier(classifier, hyperparameters, dataset, X, y, for_CV=False):
    clf_hyperparameters = copy.deepcopy(hyperparameters)
    integer_params = ['max_bin','min_data_in_leaf','num_leaves','num_iterations','hidden_layer_sizes', 'epochs', 'batch_size']
    float_params = ['bagging_fraction','feature_fraction','lambda_l1','lambda_l2','min_gain_to_split','min_sum_hessian_in_leaf']
    choice_params = {"learning_rate": ["adaptive"], # NN
                    "max_iter": [10000000], # NN
                    "tol": [1e-3], # NN
                    "input_dim": [X.shape[1]], # DNN, CNN, ResNet
                    "output_dim": [y.nunique()], # DNN, CNN, ResNet
                    "depth": [2,3,4,5,6,7,8,9,10], # DNN
                    "width": [25,50,100,150,200], # DNN, ResNet
                    "drop": [0.1, 0.01, 1e-5], # DNN, ResNet
                    "activation": ["tanh"], # NN
                    "solver": ["lbfgs","sgd","adam"], # NN
                    "batch_size": [128,256,512,1024], # NN, DNN, CNN,
                    "bagging_freq": [1, 5, 10]
                    }

    #static_params = copy.deepcopy(STATIC_PARAMS)
    static_params = {'n_jobs': -1}
    
    for k in list(clf_hyperparameters[classifier].keys()):
        if k.startswith('_'):
            del clf_hyperparameters[classifier][k]
            continue
        if np.isnan(clf_hyperparameters[classifier][k]):
            del clf_hyperparameters[classifier][k]
            continue
        if k in integer_params:
            clf_hyperparameters[classifier][k] = int(clf_hyperparameters[classifier][k])
        if k in float_params:
            clf_hyperparameters[classifier][k] = clf_hyperparameters[classifier][k]
        if k in choice_params.keys():
            clf_hyperparameters[classifier][k] = choice_params[k][int(clf_hyperparameters[classifier][k])]

    params = {**clf_hyperparameters[classifier], **static_params}
    if classifier not in ["RUMBoost", "ResLogit"]:
        base_clf = model_type_to_class[classifier](**params)
    
        return base_clf
    
    return params

## Experiment 4.1: Which is the best model?

In [22]:
## Construct Experiment 4 - Accuracy, GMPCA and Time Tables
def construct_experiment_4_accuracy_table(Experiment_4_CV_scores, Experiment_4_test_scores):
    columns = ["Accuracy", "GMPCA"]

    # Compute the mean of all the stored results for all the models and construct the final table
    train_scores_df = {}
    test_scores_df = {}
    time_scores_df = {}

    Experiment_4_CV_table_i = {}
    Experiment_4_test_table_i = {}
    Experiment_4_time_table_i = {}

    Experiment_4_CV_scores_mean = copy.deepcopy(Experiment_4_CV_scores)
    Experiment_4_test_scores_round = copy.deepcopy(Experiment_4_test_scores)
    
    for k_i in Experiment_4_CV_scores_mean.keys():
        train_scores_df[k_i] = {}
        test_scores_df[k_i] = {}
        time_scores_df[k_i] = {}
        if k_i not in ["LightGBM","NN","DNN", "RUMBoost","MNL", "ResLogit"] and k_i < 20:
            for k_clf in model_type_to_class.keys():
                for k_dataset in Experiment_4_CV_scores_mean[k_i][k_clf].keys():
                    for k_score in Experiment_4_CV_scores_mean[k_i][k_clf][k_dataset].keys():
                        if k_score in columns + ['Estimation time']:
                            Experiment_4_CV_scores_mean[k_i][k_clf][k_dataset][k_score] = np.round(np.mean(Experiment_4_CV_scores_mean[k_i][k_clf][k_dataset][k_score]), rounding)
                            Experiment_4_test_scores_round[k_i][k_clf][k_dataset][k_score] = np.round(Experiment_4_test_scores_round[k_i][k_clf][k_dataset][k_score], rounding)
                        

                train_scores_df[k_i][k_clf] = pd.DataFrame(Experiment_4_CV_scores_mean[k_i][k_clf]).T[columns]
                test_scores_df[k_i][k_clf] = pd.DataFrame(Experiment_4_test_scores_round[k_i][k_clf]).T[columns]
                time_scores_df[k_i][k_clf] = pd.DataFrame(Experiment_4_CV_scores_mean[k_i][k_clf]).T['Estimation time']
                
            Experiment_4_CV_table_i[k_i] = pd.concat(train_scores_df[k_i], axis=1)
            Experiment_4_test_table_i[k_i] = pd.concat(test_scores_df[k_i], axis=1)
            Experiment_4_time_table_i[k_i] = pd.concat(time_scores_df[k_i], axis=1)

    Experiment_4_CV_table = pd.DataFrame(pd.concat(Experiment_4_CV_table_i, axis=0).mean(axis=0)).T.set_index(pd.Index(['SwissMetro']))
    Experiment_4_test_table = pd.DataFrame(pd.concat(Experiment_4_test_table_i, axis=0).mean(axis=0)).T.set_index(pd.Index(['SwissMetro']))
    Experiment_4_time_table = pd.DataFrame(pd.concat(Experiment_4_time_table_i, axis=0).mean(axis=0)).T.set_index(pd.Index(['SwissMetro']))
    
    return (Experiment_4_CV_table, Experiment_4_test_table, Experiment_4_time_table)

## Execute the experiment

In [210]:
## Execute experiments

## Initialize dictionaries to store partial results
# Load the previous experiment data (deserialize)
try:
    with open(partial_results_dir + '/Experiment_4_CV_scores.pickle', 'rb') as handle:
        Experiment_4_CV_scores = pickle.load(handle)
except:
    Experiment_4_CV_scores = {}
try:
    with open(partial_results_dir + '/Experiment_4_test_scores.pickle', 'rb') as handle:
        Experiment_4_test_scores = pickle.load(handle)
except:
    Experiment_4_test_scores = {}

best_hyperparameters = {}

dataset = load_preprocess_SwissMetro(full_data=True)
hh_id = dataset['ID']
#dataset = dataset.drop(['ID'], axis=1)
X = dataset.drop(['choice'], axis=1)
y = dataset['choice']

dataset_name = 'SwissMetro'
dataset_id = 'swissmetro'

gssplit = GroupShuffleSplit(n_splits=n_i, test_size=0.3, random_state=1)

models_train = prepare_model([SwissMetro], [dataset])

for i, (train_i, test_i) in enumerate(gssplit.split(X, y, groups=hh_id)):
    X_train, y_train = X.iloc[train_i], y.iloc[train_i]
    X_test, y_test = X.iloc[test_i], y.iloc[test_i]
    dataset_train = dataset.iloc[train_i]
    dataset_test = dataset.iloc[test_i]
    # Obtain datasets for K-Fold cross validation (the same fold splits are used across all the iterations for all models)
    train_indices = []
    test_indices = []
    crossval_pickle_file = data_dir+dataset_id+"_crossval.pickle"
    hyperparameters_file = 'SwissMetro_hyperparameters_'+ str(n_iter) +'_'+ str(i) + '.csv'

    hh_id_train = X_train['ID']
    X_train = X_train.drop(['ID'], axis=1)
    X_test = X_test.drop(['ID'], axis=1)
    dataset_train = dataset_train.drop(['ID'], axis=1)
    dataset_test = dataset_test.drop(['ID'], axis=1)

    for (train_index, test_index) in stratified_group_k_fold(X_train, y_train, hh_id_train, k=CV, seed=1):
        train_indices.append(train_index)
        test_indices.append(test_index)


    
    warnings.filterwarnings('ignore')
    Experiment_4_CV_scores[i] = {}
    Experiment_4_test_scores[i] = {}
    best_hyperparameters = {}
    # Get results for the selected classifier
    for classifier in model_type_to_class.keys():

        experiment_time_init = time.perf_counter()
        
        if classifier not in ["MNL", "RUMBoost"]:
            print("\n--- %s" % classifier)
            time_ini = time.perf_counter()

            trials = hyperopt.Trials()
            best_classifier = hyperopt.fmin(fn=objective,
                                            space=hyperparameters[classifier],
                                            algo=hyperopt.tpe.suggest,
                                            max_evals=n_iter,
                                            trials=trials)

            
            elapsed_time = time.perf_counter() - time_ini
            print(f"Execution time: {elapsed_time}")
            
            best_hyperparameters[classifier] = best_classifier
            best_hyperparameters[classifier]['_best_loss'] = trials.best_trial["result"]["loss"]
            best_hyperparameters[classifier]['_best_GMPCA'] = np.exp(-trials.best_trial["result"]["loss"])
            best_hyperparameters[classifier]['_elapsed_time'] = elapsed_time

            # Partially store the results (the best hyperparameters)
            best_hyperparameters_df = pd.DataFrame(best_hyperparameters)
            best_hyperparameters_df.to_csv(adjusted_hyperparms_dir + hyperparameters_file, sep=',', index=True)


            print("\n\t--- {}".format(classifier))
            sys.stdout.flush()
            it_time_init = time.perf_counter()
        else:
            num_iter = 0                    
            n_epochs = 0

        # Create dictionary to store the results
        if not classifier in Experiment_4_CV_scores[i].keys():
            Experiment_4_CV_scores[i][classifier] = {}
        if not classifier in Experiment_4_test_scores[i].keys():
            Experiment_4_test_scores[i][classifier] = {}

        if recompute_Experiment_4==True or not (dataset_name in Experiment_4_CV_scores[i][classifier].keys()) or not (dataset_name in Experiment_4_test_scores[i][classifier].keys()):     # Create dictionary to store the results
            Experiment_4_CV_scores[i][classifier][dataset_name] = {}
            Experiment_4_CV_scores[i][classifier][dataset_name]['Accuracy'] = []
            Experiment_4_CV_scores[i][classifier][dataset_name]['F1'] = []
            Experiment_4_CV_scores[i][classifier][dataset_name]['Recall'] = []
            Experiment_4_CV_scores[i][classifier][dataset_name]['GMPCA'] = []
            Experiment_4_CV_scores[i][classifier][dataset_name]['Estimation time'] = []
            Experiment_4_test_scores[i][classifier][dataset_name] = {}

            ## Applying k-Fold Cross Validation over training set
            for iteration in range(0, len(train_indices)):
                print("\t\t CV it: {}".format(iteration))
                sys.stdout.flush()

                # Create the classifier
                if classifier == "RUMBoost":
                    #params_best = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train, for_CV=True)
                    params = copy.deepcopy(STATIC_PARAMS)
                    #params.update(params_best)
                    rum_structure = bio_to_rumboost(models_train[0])
                elif classifier == "ResLogit":
                    params = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train, for_CV=True)
                elif classifier == "MNL":
                    pass
                else:
                    clf = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train, for_CV=True)

                # Obtain training and testing data for this iteration (split of de k-Fold)
                X_train_CV, X_test_CV = X_train.iloc[train_indices[iteration]], X_train.iloc[test_indices[iteration]]
                y_train_CV, y_test_CV = y_train.iloc[train_indices[iteration]], y_train.iloc[test_indices[iteration]]
                
                dataset_train_CV = dataset_train.iloc[train_indices[iteration]]
                dataset_test_CV = dataset_train.iloc[test_indices[iteration]]

                # Scale the data
                scaler = StandardScaler()
                scaler.fit(X_train_CV[scaled_fetures])
                X_train_CV.loc[:, scaled_fetures] = scaler.transform(X_train_CV[scaled_fetures])
                X_test_CV.loc[:, scaled_fetures] = scaler.transform(X_test_CV[scaled_fetures])
                dataset_train_CV.loc[:, scaled_fetures] = scaler.transform(dataset_train_CV[scaled_fetures])
                dataset_test_CV.loc[:, scaled_fetures] = scaler.transform(dataset_test_CV[scaled_fetures])

                train_set = lgb.Dataset(X_train_CV, label=y_train_CV, free_raw_data=False)
                test_set = lgb.Dataset(X_test_CV, label=y_test_CV, free_raw_data=False)

                # Balance dataset
                #X_train, y_train = balance(X_train, y_train, X_train.shape[0], len(dataset["alt_names"]))
                
                time_ini = time.perf_counter()
                if classifier == "RUMBoost":
                    clf_trained = rum_train(params, train_set, rum_structure, valid_sets=[test_set])
                    num_iter += clf_trained.best_iteration
                elif classifier == "MNL":
                    models_train = prepare_model([SwissMetro_normalised], [dataset_train_CV])
                    clf_trained = estimate_models(models_train)
                elif classifier == "ResLogit":
                    clf = ResLogit(X_train_CV, y_train_CV, params["input_dim"], params["output_dim"], n_layers=params['depth'], batch_size=params['batch_size'], epochs=params['epochs'])
                    _, epochs, proba = clf.train(X_train_CV, y_train_CV, X_test_CV, y_test_CV)
                    n_epochs += epochs
                else:
                    clf.fit(X_train_CV, y_train_CV)
                elapsed_time = time.perf_counter() - time_ini

                if classifier == "MNL":
                    labels = prepare_labels([dataset_test_CV])
                    model_test = prepare_model([SwissMetro_normalised], [dataset_test_CV], for_prob=True)
                    proba = predict_test(clf_trained, model_test, labels, return_prob=True)
                    y_score = np.argmax(proba, axis=1)
                elif classifier == "RUMBoost":
                    proba = clf_trained.predict(test_set)
                    y_score = np.argmax(proba, axis=1)
                elif classifier == "ResLogit":
                    y_score = np.argmax(proba, axis=1)
                else:
                    proba = clf.predict_proba(X_test_CV)
                    y_score = np.argmax(proba, axis=1)

                # Compute the accuracy results
                Experiment_4_CV_scores[i][classifier][dataset_name]['Accuracy'] = np.append(Experiment_4_CV_scores[i][classifier][dataset_name]['Accuracy'], accuracy_score(y_test_CV, y_score)*100)
                Experiment_4_CV_scores[i][classifier][dataset_name]['F1'] = np.append(Experiment_4_CV_scores[i][classifier][dataset_name]['F1'], f1_score(y_test_CV, y_score, average=average_tech)*100)
                Experiment_4_CV_scores[i][classifier][dataset_name]['Recall'] = np.append(Experiment_4_CV_scores[i][classifier][dataset_name]['Recall'], recall_score(y_test_CV, y_score, average=average_tech)*100)
                Experiment_4_CV_scores[i][classifier][dataset_name]['GMPCA'] = np.append(Experiment_4_CV_scores[i][classifier][dataset_name]['GMPCA'], GMPCA(proba, y_test_CV.values)*100)
                Experiment_4_CV_scores[i][classifier][dataset_name]['Estimation time'] = np.append(Experiment_4_CV_scores[i][classifier][dataset_name]['Estimation time'], elapsed_time)

                if classifier in ["MNL", "RUMBoost"]:
                    del clf_trained
                else:
                    del clf
                gc.collect()

            ## Out-of-sample results
            # Create the classifier
            if classifier == "RUMBoost":
                #params_best = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train, for_CV=True)
                params = copy.deepcopy(STATIC_PARAMS)
                #params.update(params_best)
                params['num_iterations'] = int(num_iter/CV)
                rum_structure = bio_to_rumboost(models_train[0])
            elif classifier == "ResLogit":
                params = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train, for_CV=True)
                opt_epochs = int(n_epochs/CV)
            elif classifier == "MNL":
                pass
            else:
                clf = create_classifier(classifier, best_hyperparameters, dataset, X_train, y_train,)
            fitted = True

            # Scale the data
            scaler = StandardScaler()
            scaler.fit(X_train[scaled_fetures])
            X_scaled = X_train.copy()
            final_test_X_scaled = X_test.copy()
            X_scaled.loc[:, scaled_fetures] = scaler.transform(X_scaled[scaled_fetures])
            final_test_X_scaled.loc[:, scaled_fetures] = scaler.transform(final_test_X_scaled[scaled_fetures])

            train_set = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
            test_set = lgb.Dataset(X_test, label=y_test, free_raw_data=False)

            # Balance dataset
            #X_scaled_balanced, y_balanced = balance(X_scaled, y, X_scaled.shape[0], len(dataset["alt_names"]))

            # Fit the classifier on training set
            time_ini = time.perf_counter()
            if classifier == "RUMBoost":
                clf_trained = rum_train(params, train_set, rum_structure, valid_sets=[train_set])
            elif classifier == "MNL":
                models_train = prepare_model([SwissMetro_normalised], [dataset_train])
                clf_trained = estimate_models(models_train)
            elif classifier == "ResLogit":
                clf = ResLogit(X_scaled, y_train, params["input_dim"], params["output_dim"], n_layers=params['depth'], batch_size=params['batch_size'], epochs=opt_epochs)
                _, _, _ = clf.train(X_train_CV, y_train_CV, None, None)
            else:
                clf.fit(X_scaled, y_train)
            elapsed_time = time.perf_counter() - time_ini

            if classifier == "MNL":
                labels = prepare_labels([dataset_test])
                model_test = prepare_model([SwissMetro_normalised], [dataset_test], for_prob=True)
                proba = predict_test(clf_trained, model_test, labels, return_prob=True)
                y_score = np.argmax(proba, axis=1)
            elif classifier == "RUMBoost":
                proba = clf_trained.predict(test_set)
                y_score = np.argmax(proba, axis=1)
            elif classifier == "ResLogit":
                proba = clf.predict_validate(final_test_X_scaled)
                y_score = np.argmax(proba, axis=1)
            else:
                proba = clf.predict_proba(final_test_X_scaled)
                y_score = np.argmax(proba, axis=1)

            fitted = True

            if classifier == "RUMBoost":
                clf_trained.save_model(partial_results_dir + f"SwissMetro_RUMBoost_{i}_test.json")
                
            # Compute the accuracy results
            Experiment_4_test_scores[i][classifier][dataset_name]['Accuracy'] = accuracy_score(y_test, y_score)*100
            Experiment_4_test_scores[i][classifier][dataset_name]['F1'] = f1_score(y_test, y_score, average=average_tech)*100
            Experiment_4_test_scores[i][classifier][dataset_name]['Recall'] = recall_score(y_test, y_score, average=average_tech)*100
            Experiment_4_test_scores[i][classifier][dataset_name]['GMPCA'] = GMPCA(proba, y_test.values)*100
            Experiment_4_test_scores[i][classifier][dataset_name]['Estimation time'] = elapsed_time

            ## Market shares
            #Experiment_4_CV_scores[classifier][dataset_name]['Market_shares'] = np.round(np.sum(clf.predict_proba(X_scaled), axis=0)/X_scaled.shape[0] * 100, 3)
            #Experiment_4_test_scores[classifier][dataset_name]['Market_shares'] = np.round(np.sum(clf.predict_proba(final_test_X_scaled), axis=0)/final_test_X_scaled.shape[0] * 100, 3)
            
            ## WTP
            #Experiment_4_CV_scores[classifier][dataset_name]["WTP_history"] = None
            #Experiment_4_test_scores[classifier][dataset_name]["WTP_history"] = None
            # if dataset["WTP"] is not None:
            #     Experiment_4_CV_scores[classifier][dataset_name]["WTP_history"] = {}
            #     Experiment_4_CV_scores[classifier][dataset_name]["n_WTP_nan"] = 0
            #     Experiment_4_CV_scores[classifier][dataset_name]["n_WTP_inf"] = 0
            #     Experiment_4_test_scores[classifier][dataset_name]["WTP_history"] = {}
            #     Experiment_4_test_scores[classifier][dataset_name]["n_WTP_nan"] = 0
            #     Experiment_4_test_scores[classifier][dataset_name]["n_WTP_inf"] = 0

            #     for alt in dataset["WTP"].keys():
            #         v1_name, v2_name, d_1, d_2 = dataset["WTP"][alt]

            #         # WTP over training set 
            #         filtered_WTP, n_WTP_nan, n_WTP_inf = compute_WTP(clf, dataset, X, v1_name, v2_name, d_1, d_2, scaler)
            #         Experiment_4_CV_scores[classifier][dataset_name]["n_WTP_nan"] += n_WTP_nan
            #         Experiment_4_CV_scores[classifier][dataset_name]["n_WTP_inf"] += n_WTP_inf
            #         Experiment_4_CV_scores[classifier][dataset_name]["WTP_history"][dataset["alt_names"][alt]] = filtered_WTP

            #         # WTP over test set 
            #         filtered_WTP, n_WTP_nan, n_WTP_inf = compute_WTP(clf, dataset, final_test_X, v1_name, v2_name, d_1, d_2, scaler)
            #         Experiment_4_test_scores[classifier][dataset_name]["n_WTP_nan"] += n_WTP_nan
            #         Experiment_4_test_scores[classifier][dataset_name]["n_WTP_inf"] += n_WTP_inf
            #         Experiment_4_test_scores[classifier][dataset_name]["WTP_history"][dataset["alt_names"][alt]] = filtered_WTP

            if classifier in ["MNL", "RUMBoost"]:
                del clf_trained
            else:
                del clf
            gc.collect()

        # Store the partial experiment data (serialize)
        with open(partial_results_dir + 'Experiment_4_CV_scores.pickle', 'wb') as handle:
            pickle.dump(Experiment_4_CV_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(partial_results_dir + 'Experiment_4_test_scores.pickle', 'wb') as handle:
            pickle.dump(Experiment_4_test_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

        print("\t    + Elapsed: {} seconds".format(np.round(time.perf_counter()-experiment_time_init), 2))


		 CV it: 0
Early stopping at iteration 317, with a best score on test set of 0.7522265353875153, and on train set of 0.6819037344954868
Finished loading model, total used 418 iterations
Finished loading model, total used 418 iterations
Finished loading model, total used 418 iterations
		 CV it: 1
Early stopping at iteration 358, with a best score on test set of 0.7177982510317341, and on train set of 0.6886690331163347
Finished loading model, total used 459 iterations
Finished loading model, total used 459 iterations
Finished loading model, total used 459 iterations
		 CV it: 2
Early stopping at iteration 322, with a best score on test set of 0.7499931855658396, and on train set of 0.6800689116174075
Finished loading model, total used 423 iterations
Finished loading model, total used 423 iterations
Finished loading model, total used 423 iterations
		 CV it: 3
Early stopping at iteration 680, with a best score on test set of 0.7133537632333002, and on train set of 0.677268262728821
Fin

## Export LaTeX tables/figures

In [37]:
try:
    with open(partial_results_dir + '/Experiment_4_CV_scores.pickle', 'rb') as handle:
        Experiment_4_CV_scores = pickle.load(handle)
except:
    Experiment_4_CV_scores = {}
try:
    with open(partial_results_dir + '/Experiment_4_test_scores.pickle', 'rb') as handle:
        Experiment_4_test_scores = pickle.load(handle)
except:
    Experiment_4_test_scores = {}

In [38]:
Experiment_4_test_scores

{'LightGBM': {'LPMC': {'Accuracy': 74.76443768996961,
   'F1': 55.77533290152894,
   'Recall': 55.86976689126527,
   'GMPCA': 52.00953889879196,
   'Estimation time': 5.394302400000015,
   'Market_shares': array([17.001,  2.857, 36.225, 43.917]),
   'WTP_history': None}},
 'NN': {'LPMC': {'Accuracy': 74.22492401215806,
   'F1': 55.252585435855494,
   'Recall': 55.510437716485306,
   'GMPCA': 51.34357600821821,
   'Estimation time': 11.689457000000175,
   'Market_shares': array([16.941,  2.9  , 36.089, 44.069]),
   'WTP_history': None}},
 'DNN': {'LPMC': {'Accuracy': 74.36170212765958,
   'F1': 55.48164394231152,
   'Recall': 55.92791501690747,
   'GMPCA': 50.99265789019044,
   'Estimation time': 4.8106472999998005,
   'Market_shares': array([17.711,  2.784, 35.031, 44.474], dtype=float32),
   'WTP_history': None}},
 0: {'RUMBoost': {'SwissMetro': {'Accuracy': 67.99116997792495,
    'F1': 49.43119812699326,
    'Recall': 49.125855481671614,
    'GMPCA': 46.97805026510724,
    'Estimatio

In [39]:
model_type_to_class = {#"LightGBM": LightGBM,
                       #"NN": NN,
                       #"DNN": DNN,
                       "RUMBoost": "RUMBoost",
                       #"MNL":"MNL",
                       #"ResLogit":"ResLogit",
                      }

In [40]:
# Obtain accuracy tables
Experiment_4_CV_table, Experiment_4_test_table, Experiment_4_time_table = construct_experiment_4_accuracy_table(Experiment_4_CV_scores, Experiment_4_test_scores)

# Obtain market shares table over the test set
#Experiment_4_MS_table = construct_experiment_4_market_shares_table(Experiment_4_test_scores, datasets)

In [41]:
Experiment_4_CV_table

RUMBoost        
           Accuracy   GMPCA
SwissMetro   67.463  47.868

In [42]:
Experiment_4_test_table

RUMBoost        
           Accuracy   GMPCA
SwissMetro   67.277  47.524

In [43]:
Experiment_4_time_table

,RUMBoost
SwissMetro,0.4145


In [48]:
model_type_to_class = {"LightGBM": LightGBM,
                       "NN": NN,
                       "DNN": DNN,
                       #"RUMBoost": "RUMBoost",
                       "MNL":"MNL",
                       #"ResLogit":"ResLogit",
                      }

In [49]:
try:
    with open(partial_results_dir + '/Experiment_4_CV_scores_50_all.pickle', 'rb') as handle:
        Experiment_4_CV_scores = pickle.load(handle)
except:
    Experiment_4_CV_scores = {}
try:
    with open(partial_results_dir + '/Experiment_4_test_scores_50_all.pickle', 'rb') as handle:
        Experiment_4_test_scores = pickle.load(handle)
except:
    Experiment_4_test_scores = {}

In [50]:
Experiment_4_CV_scores

{'LightGBM': {'LPMC': {'Accuracy': array([76.33741099, 76.53825087, 74.55723936, 75.49529809, 75.61866496]),
   'F1': array([56.85988533, 57.34935852, 55.26020222, 56.47404185, 56.09915072]),
   'Recall': array([56.99342923, 57.33803397, 55.59171484, 56.6973145 , 56.20315315]),
   'GMPCA': array([53.65254338, 53.30342893, 52.06810003, 52.64873025, 52.46773231]),
   'Estimation time': array([4.6611866, 4.6994733, 4.6744622, 4.4110102, 4.7360106]),
   'Market_shares': array([17.495,  2.815, 34.888, 44.802]),
   'WTP_history': None}},
 'NN': {'LPMC': {'Accuracy': array([74.7215629 , 75.68924594, 74.1646887 , 74.97489272, 74.29458497]),
   'F1': array([55.79286181, 56.3438783 , 55.59670061, 55.91607256, 54.99354968]),
   'Recall': array([55.98365431, 56.60842644, 55.80929466, 56.33617128, 55.20810395]),
   'GMPCA': array([52.60944677, 52.62503945, 51.60453968, 52.04425195, 51.73404377]),
   'Estimation time': array([7.9773714, 7.8353275, 7.6807843, 8.0482044, 7.7013456]),
   'Market_shares

In [51]:
# Obtain accuracy tables
Experiment_4_CV_table, Experiment_4_test_table, Experiment_4_time_table = construct_experiment_4_accuracy_table(Experiment_4_CV_scores, Experiment_4_test_scores)

# Obtain market shares table over the test set
#Experiment_4_MS_table = construct_experiment_4_market_shares_table(Experiment_4_test_scores, datasets)

In [52]:
Experiment_4_CV_table

LightGBM               NN              DNN              MNL  \
           Accuracy   GMPCA Accuracy   GMPCA Accuracy   GMPCA Accuracy   
SwissMetro  69.5305  49.611  68.3465  47.852    68.67  48.143  66.8965   

                     
              GMPCA  
SwissMetro  45.7615

In [53]:
Experiment_4_test_table

LightGBM               NN              DNN               MNL  \
           Accuracy   GMPCA Accuracy   GMPCA Accuracy    GMPCA Accuracy   
SwissMetro  69.3505  49.162  68.1595  47.869  68.5665  47.4355  67.3165   

                     
              GMPCA  
SwissMetro  45.4205

In [54]:
Experiment_4_time_table

,LightGBM,NN,DNN,MNL
SwissMetro,0.6525,1.714,2.7995,1.022


RUMBoost PCUF on same test set: 45.79

## Swissmetro MNL

In [7]:
dataset = load_preprocess_SwissMetro(full_data=True)
models = prepare_model([SwissMetro_normalised], [dataset])

results = estimate_models(models)[0]



Results for model SwissmetroMNL
Nbr of parameters:		17
Sample size:			9036
Excluded data:			0
Final log likelihood:		-6914.205
Akaike Information Criterion:	13862.41
Bayesian Information Criterion:	13983.26

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR     -4.848136      0.350495   -13.832239  0.000000e+00
ASC_TRAIN   -0.547535      0.083859    -6.529238  6.610512e-11
B_AGE_1      0.775633      0.201896     3.841747  1.221619e-04
B_AGE_2      0.161117      0.055684     2.893421  3.810700e-03
B_COST      -0.009325      0.000479   -19.468700  0.000000e+00
B_FIRST     -0.337905      0.077237    -4.374928  1.214727e-05
B_HE        -0.006985      0.001036    -6.744933  1.530975e-11
B_MALE      -0.282183      0.065905    -4.281631  1.855282e-05
B_PURPOSE_1 -3.816300      0.357925   -10.662288  0.000000e+00
B_PURPOSE_2 -3.242224      0.371445    -8.728668  0.000000e+00
B_PURPOSE_3 -4.528800      0.352723   -12.839550  0.000000e+00
B_PURPOSE_4 -5.274692      0.352474 

In [12]:
results.getEstimatedParameters().to_csv('../Data/Results-RealDatasets/swissmetro_parameters.csv')

In [13]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,-4.848136,0.350495,-13.832239,0.000000e+00
ASC_TRAIN,-0.547535,0.083859,-6.529238,6.610512e-11
B_AGE_1,0.775633,0.201896,3.841747,1.221619e-04
B_AGE_2,0.161117,0.055684,2.893421,3.810700e-03
B_COST,-0.009325,0.000479,-19.468700,0.000000e+00
B_FIRST,-0.337905,0.077237,-4.374928,1.214727e-05
B_HE,-0.006985,0.001036,-6.744933,1.530975e-11
B_MALE,-0.282183,0.065905,-4.281631,1.855282e-05
B_PURPOSE_1,-3.816300,0.357925,-10.662288,0.000000e+00
B_PURPOSE_2,-3.242224,0.371445,-8.728668,0.000000e+00
